In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
data = pd.read_csv('data/data.csv')

In [4]:
data.dtypes

id            float64
date_start     object
contracts      object
dtype: object

In [5]:
def parse_contract_date(contract_date_str, application_date):
    if contract_date_str:
        contract_date = pd.to_datetime(contract_date_str, format="%d.%m.%Y")
        if contract_date > application_date:
            return application_date
        else:
            return contract_date
    else:
        return None

In [6]:
def calculate_number_of_claims_last_180_days(contracts, application_date):
    if contracts is np.NaN:
        return -3
    contracts = eval(contracts)
    if type(contracts) != list:
      contracts = [contracts]
    valid_contracts = [contract for contract in contracts if contract['claim_date']]
    valid_contracts_last_180_days = [contract for contract in valid_contracts
                                     if (application_date - pd.to_datetime(contract['claim_date'], format="%d.%m.%Y")).days <= 180]
    return len(valid_contracts_last_180_days)


In [7]:
def calculate_exposure_of_active_bank_loans(contracts):

    if contracts is np.NaN:
        return -3
    contracts = eval(contracts)
    if type(contracts) != list:
      contracts = [contracts]
    valid_contracts = [contract for contract in contracts
                       if (contract['bank'] not in ['LIZ', 'LOM', 'MKO', 'SUG', None] if 'bank' in contract.keys() else False) and contract['loan_summa']
                      ]
    disbursed_loans = [contract for contract in valid_contracts if contract['contract_date']]
    exposure = sum(float(contract['loan_summa']) for contract in disbursed_loans)
    return exposure

In [8]:
def calculate_days_since_last_loan(contracts, application_date):
    if contracts is np.NaN:
        return -3
    contracts = eval(contracts)
    if type(contracts) != list:
      contracts = [contracts]
    valid_contracts = [contract for contract in contracts if contract['summa']]
    if not valid_contracts:
        return -1
    sorted_contracts = sorted(valid_contracts, key=lambda x: parse_contract_date(x['contract_date'], application_date))
    last_contract = sorted_contracts[-1]
    contract_date = parse_contract_date(last_contract['contract_date'], application_date)
    days_since_last_loan = (application_date - contract_date).days
    return days_since_last_loan

In [9]:
application_date = pd.to_datetime('today')

data['tot_claim_cnt_l180d'] = data['contracts'].apply(lambda x: calculate_number_of_claims_last_180_days(x, application_date))
data['disb_active_bank_loan_wo_tbc'] = data['contracts'].apply(calculate_exposure_of_active_bank_loans)
data['day_sinlastloan'] = data['contracts'].apply(lambda x: calculate_days_since_last_loan(x, application_date))

In [10]:
data.drop('contracts', axis=1)

,id,date_start,tot_claim_cnt_l180d,disb_active_bank_loan_wo_tbc,day_sinlastloan
0,2925210.0,2024-02-12 19:22:46.652000+00:00,-3,-3.0,-3
1,2925211.0,2024-02-12 19:24:29.135000+00:00,58,0.0,441
2,2925212.0,2024-02-12 19:24:41.493000+00:00,-3,-3.0,-3
3,2925213.0,2024-02-12 19:24:29.135000+00:00,58,0.0,441
4,2925214.0,2024-02-12 19:24:56.857000+00:00,-3,-3.0,-3
...,...,...,...,...,...
995,2926205.0,2024-02-13 06:09:54.210000+00:00,0,217320883.0,742
996,2926206.0,2024-02-13 06:09:54.306000+00:00,-3,-3.0,-3
997,2926207.0,2024-02-13 06:09:55.661000+00:00,-3,-3.0,-3
998,2926208.0,2024-02-13 06:09:57.024000+00:00,-3,-3.0,-3


In [18]:
data.to_csv('features.csv')